In [35]:
!pip install transformers torch langchain rank-bm25 wikipedia
!pip install rank_bm25 nltk


In [36]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

import plotly.graph_objects as go
import plotly.express as px


import warnings
warnings.simplefilter(action='ignore', category=Warning)

import os

import json
import re
import nltk
from rank_bm25 import BM25Okapi
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Preprocessing

(NOT USED, simple implementation used earlier) simply find keywords in title

In [37]:
import json
import re

# Load the JSON file
with open('/kaggle/input/intern-json/news.article.json', 'r') as file:
    articles = json.load(file)

# Function to clean the text
def clean_text(text):
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation and special characters
    return text.strip()


In [38]:
# Clean the articles
cleaned_articles = []
for article in articles:
    if 'Israel' in article['title'] or 'Hamas' in article['title']:
        cleaned_articles.append({
            'title': article['title'],
            'articleBody': clean_text(article['articleBody'])
        })

In [39]:
cleaned_articles[25]

{'title': 'Israel Hamas War: French President Emmanuel Macron Visits Israel, Meets President Isaac Herzog',
 'articleBody': 'Israel Hamas War French President Emmanuel Macron on meeting with Israeli President Isaac Herzog in Tel Aviv today said that he is there to express support and solidarity What happened on October 7 was a terrorist attack against your people and your nation he said French President Emmanuel Macron arrived in Tel Aviv on Tuesday to express his countrys full solidarity with Israel after the deadly October 7 attacks by Palestinian militant group Hamas The war is the deadliest of five Gaza wars for both sides The Hamasrun Health Ministry said Monday that at least 5087 Palestinians have been killed and 15270 wounded In the occupied West Bank 96 Palestinians have been killed and 1650 wounded in violence and Israeli raids since Oct 7 More than 1400 people in Israel have been killed'}

In [40]:
# column names are as follow, [articleBody,	dateModified,	scrapedDate,	source,	title].

# Document Retrieval: Top 5 Most Relevent

1. The preprocess function cleans and tokenizes the text of each article.
2. The BM25Okapi model is initialized with the tokenized articles.
3. The retrieve_documents function scores each article based on its relevance to the query using BM25 and returns the top N articles.

In [41]:
# Download required NLTK data files
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [42]:
# Function to clean and tokenize text
def preprocess(text):
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation and special characters
    text = text.replace('’', " ")  # Replace smart quotes with standard quotes
    text = text.replace('“', '(')  # Replace other invalid characters as needed
    text = text.replace('”', ')')
    text = text.replace('—', '-')
    tokens = nltk.word_tokenize(text.lower())
    return tokens

def retrieve_documents(query, bm25, articles, n=5):
    tokenized_query = preprocess(query)
    doc_scores = bm25.get_scores(tokenized_query)
    top_n_indices = sorted(range(len(doc_scores)), key=lambda i: doc_scores[i], reverse=True)[:n]
    return [articles[i] for i in top_n_indices]



def extract_keywords(question):
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(question)
    keywords = [word.lower() for word in words if word.isalnum() and word.lower() not in stop_words]
    return keywords

In [43]:
import spacy
nlp = spacy.load('en_core_web_sm')

def get_relevant_article(question):
    print("getting relevent document")
#     # Clean and tokenize the articles
#     tokenized_articles = [preprocess(article['articleBody']) for article in articles if 'Israel' in article['articleBody'] or 'Hamas' in article['articleBody']]

    keywords = extract_keywords(question)
#     print(keywords)
# #     keywords = ['hamas', 'israel'] + keywords  # Add 'hamas' and 'israel' to keywords
#     tokenized_articles = [preprocess(article['articleBody']) for article in articles if any(keyword in article['articleBody'] for keyword in keywords)]
#     print(tokenized_articles)
    
    
    filtered_articles = [article for article in articles if 'Israel' in article['title'] or 'Hamas' in article['title']]

    # Apply keyword search to the filtered articles
    tokenized_articles = [preprocess(article['articleBody']) for article in filtered_articles if any(keyword in article['articleBody'].lower() for keyword in keywords)]
 
    # Initialize BM25
    bm25 = BM25Okapi(tokenized_articles)

    # Example question
#     question = "What happened at the Al-Shifa Hospital?"

    # Retrieve top 5 relevant articles
    top_articles = retrieve_documents(question, bm25, articles, n=5)

    # Print the titles of the top articles
    for i, article in enumerate(top_articles):
        print(f"Article {i+1}: {article['title']}")

    # Choose the most relevant article (first one)
    relevant_article = top_articles[0]
    print("relevant_article returned - Article Length")
    return relevant_article

# Question Answering

In [44]:
from transformers import BertForQuestionAnswering, BertTokenizer
import torch
import warnings
warnings.simplefilter("ignore")


# Load pre-trained BERT model and tokenizer
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')


# weight_path = "kaporter/bert-base-uncased-finetuned-squad"
# # loading tokenizer
# tokenizer = BertTokenizer.from_pretrained(weight_path)
# #loading the model
# model = BertForQuestionAnswering.from_pretrained(weight_path)

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [45]:
def answer_question(question, context):
    
    print("answering question - ")
    # Encode the question and context separately
    

    input_ids = tokenizer.encode(question, context)
    print (f'We have about {len(input_ids)} tokens generated')
    
    if len(input_ids) < 512:
        tokens = tokenizer.convert_ids_to_tokens(input_ids)

        #first occurence of [SEP] token
        sep_idx = input_ids.index(tokenizer.sep_token_id)
        #print("SEP token index: ", sep_idx)

        num_seg_a = sep_idx+1                           #number of tokens in segment A (question) - this will be one more than the sep_idx as the index in Python starts from 0
        num_seg_b = len(input_ids) - num_seg_a          #number of tokens in segment B (text)

        #creating the segment ids
        segment_ids = [0]*num_seg_a + [1]*num_seg_b
        
        #making sure that every input token has a segment id
        assert len(segment_ids) == len(input_ids)

        output = model(torch.tensor([input_ids]),  token_type_ids=torch.tensor([segment_ids]))

        #tokens with highest start and end scores
        answer_start = torch.argmax(output.start_logits)
        answer_end = torch.argmax(output.end_logits)
        if answer_end >= answer_start:
            answer = " ".join(tokens[answer_start:answer_end+1])
        else:
            print("I am unable to find the answer to this question. Can you please ask another question?")
        return answer

    else:
        answer = 'High token length, Sorry'
        return answer

In [46]:
question = 'Number of displaced Palestinians'
relevant_article = get_relevant_article(question)
#         print(relevant_article)
answer = answer_question( question,relevant_article['articleBody'])
print("Answer - ", answer)
print()
print()
question = 'What happened at the Al-Shifa Hospital?'
relevant_article = get_relevant_article(question)
#         print(relevant_article)
answer = answer_question( question,relevant_article['articleBody'])
print("Answer - ", answer)
print()
print()

getting relevent document


Token indices sequence length is longer than the specified maximum sequence length for this model (599 > 512). Running this sequence through the model will result in indexing errors


Article 1: Wounded Israeli tattoo artist held hostage by Hamas was operated on by a vet in Gaza, her family says
Article 2: Britain Condemns Iran’s Role In Red Sea Attacks
Article 3: COP28 Day 5 Recap: President Al Jaber's 'Fossil Fuel Science' remark, green finance and 'more declarations'
Article 4: Israel Asks India To Declare Hamas as a Terrorist Organisation
Article 5: Bank Of Israel Says In View Of The War, The Monetary Committee's…
relevant_article returned - Article Length
answering question - 
We have about 599 tokens generated
Answer -  High token length, Sorry


getting relevent document
Article 1: Gaza: College Students Walk Out Calling for Israel Ceasefire
Article 2: UN Security Council fails again to address Israel-Hamas war, rejecting US and Russian resolutions
Article 3: Pentagon says a US warship commercial ships attacked in Red Sea. Houthis claim attacking 2 ships
Article 4: People support for Palestine can be seen after referendums
Article 5: Hezbollah and Israeli tro

In [48]:
while True:
    user_input = input("Enter your question ('exit' to quit): ")
    # 1.  Number of displaced Palestinians 
    # 2.  What happened at the Al-Shifa Hospital?
    if user_input.lower() == 'exit':
        break
    else:
        question = user_input
        relevant_article = get_relevant_article(question)
#         print(relevant_article)
        answer = answer_question( question,relevant_article['articleBody'])
        print("Answer - ", answer)
        print()
        print()

Enter your question ('exit' to quit):  exit
